In [10]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

No GPU was detected. CNNs can be very slow without a GPU.


In [11]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

X_train = (X_train - X_train.mean().astype(np.float32)) / X_train.std().astype(np.float32)
X_test = (X_test - X_test.mean().astype(np.float32)) / X_test.std().astype(np.float32)

X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [40]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from tensorflow.keras import layers
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPool2D, Dropout, Flatten,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

## training the model

In [28]:
epochs = 25
annealer = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.95 ** (x+epochs))

In [33]:
early_stopping_cb = keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor='val_loss',
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-3,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=25,
        verbose=1)

In [43]:

keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
model = Sequential()
model.add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(28,28,1)))
model.add(BatchNormalization())
model.add(Conv2D(32,kernel_size=3,activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(64,kernel_size=3,activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64,kernel_size=3,activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])

model.fit(X_train, y_train, epochs=25, validation_data=(X_valid, y_valid), callbacks = [early_stopping_cb,annealer])

Epoch 1/25
1719/1719 [==============================] - 120s 70ms/step - loss: 0.8287 - accuracy: 0.7446 - val_loss: 0.1185 - val_accuracy: 0.9626 - lr: 7.6945e-05
Epoch 2/25
1719/1719 [==============================] - 118s 68ms/step - loss: 0.2046 - accuracy: 0.9382 - val_loss: 0.0588 - val_accuracy: 0.9818 - lr: 7.3098e-05
Epoch 3/25
1719/1719 [==============================] - 118s 69ms/step - loss: 0.1244 - accuracy: 0.9623 - val_loss: 0.0417 - val_accuracy: 0.9890 - lr: 6.9443e-05
Epoch 4/25
1719/1719 [==============================] - 117s 68ms/step - loss: 0.0901 - accuracy: 0.9724 - val_loss: 0.0335 - val_accuracy: 0.9908 - lr: 6.5971e-05
Epoch 5/25
1719/1719 [==============================] - 117s 68ms/step - loss: 0.0725 - accuracy: 0.9779 - val_loss: 0.0294 - val_accuracy: 0.9908 - lr: 6.2672e-05
Epoch 6/25
1719/1719 [==============================] - 125s 72ms/step - loss: 0.0631 - accuracy: 0.9814 - val_loss: 0.0285 - val_accuracy: 0.9912 - lr: 5.9539e-05
Epoch 7/25
1719/

In [44]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 3s 10ms/step - loss: 0.0158 - accuracy: 0.9949


[0.01576046831905842, 0.9948999881744385]